In [1]:
using Distributions, LinearAlgebra, Statistics, StatsFuns
cd("../src/")
include("hcmvn.jl")
include("hchol.jl")
include("generate.jl")

Σ_morton_generate

In [2]:
iters = 20
# ns = [512, 1024, 2048]
ns = [4^2, 8^2, 16^2]
m = 8
βs = [0.3, 0.1, 0.03];

In [3]:
# beta1
ans_arr = zeros(length(ns), length(βs), iters, 5)
time_arr = zeros(length(ns), length(βs), 5)
real_arr = zeros(length(ns), length(βs), iters)
h_size_arr = zeros(length(ns), length(βs), iters);

In [4]:
for i in 1:length(ns)
    n = ns[i]
    for p in 1:iters
        for j in 1:length(βs)
            Σ_morton = Σ_morton_generate(n, βs[j])
            a = fill(-Inf, n)
            b = rand(Uniform(0, n), n)
            
            B, UV = hchol(Σ_morton, m)
            h_size_arr[i, j, p] += (Base.summarysize(UV) + Base.summarysize(B)) / Base.summarysize(Σ_morton)
        
            time_arr[i, j, 1] += @elapsed ans_arr[i, j, p, 1] += mvn(cholesky(Σ_morton).L, a, b, ns = 5, N = 1000)
            time_arr[i, j, 2] += @elapsed ans_arr[i, j, p, 2] += HMVN(a, b, Σ_morton, m, ns = 5, N = 1000)
            time_arr[i, j, 3] += @elapsed ans_arr[i, j, p, 3] += HCMVN(a, b, Σ_morton, 1, m, ns = 5, N = 1000)
            time_arr[i, j, 4] += @elapsed ans_arr[i, j, p, 4] += HCMVN(a, b, Σ_morton, 2, m, ns = 5, N = 1000)
            time_arr[i, j, 5] += @elapsed ans_arr[i, j, p, 5] += HCMVN(a, b, Σ_morton, 4, m, ns = 5, N = 1000)
            
            real_arr[i, j, p] += mvn(cholesky(Σ_morton).L, a, b, ns = 10, N = 2000)
        end
    end
end

In [5]:
h_size_arr

3×3×20 Array{Float64,3}:
[:, :, 1] =
 0.885932  0.885932  0.885932
 0.559834  0.559834  0.419449
 0.277207  0.277207  0.199091

[:, :, 2] =
 0.885932  0.885932  0.885932
 0.559834  0.559834  0.419449
 0.277207  0.277207  0.199091

[:, :, 3] =
 0.885932  0.885932  0.885932
 0.559834  0.559834  0.419449
 0.277207  0.277207  0.199091

...

[:, :, 18] =
 0.885932  0.885932  0.885932
 0.559834  0.559834  0.419449
 0.277207  0.277207  0.199091

[:, :, 19] =
 0.885932  0.885932  0.885932
 0.559834  0.559834  0.419449
 0.277207  0.277207  0.199091

[:, :, 20] =
 0.885932  0.885932  0.885932
 0.559834  0.559834  0.419449
 0.277207  0.277207  0.199091

In [6]:
time_arr

3×3×5 Array{Float64,3}:
[:, :, 1] =
  1.72072   0.273753   0.2772 
  1.43788   1.4349     1.45798
 14.8912   14.7513    14.9451 

[:, :, 2] =
  4.10774   2.32233   2.32195
  8.43881   8.41597   8.61391
 30.1833   30.1243   30.1369 

[:, :, 3] =
 0.921364   0.00899981  0.00874537
 0.0520604  0.0500322   0.0441492 
 0.262972   0.295667    0.249295  

[:, :, 4] =
  0.761298   0.750492   0.753395
  2.76687    2.78241    2.83633 
 10.4144    10.3822    10.3385  

[:, :, 5] =
  1.60064   1.57894   1.56304
  5.54095   5.54977   5.60384
 20.5864   20.652    20.452  

In [7]:
sum(time_arr)

302.6620518219999

In [8]:
using JLD2, FileIO
cd("../tables")

In [10]:
@save "table5.jld2" ans_arr time_arr real_arr h_size_arr iters